In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from torchmetrics import Accuracy
import torch.nn.init as init
import pandas as pd
import numpy as np

# 更改工作目录
os.chdir('/home/ubuntu/zlb/MEIJU/train_model')

import config as cf

# 定义的模型
from transformer_model import MultimodalDataset,MultimodalEmotionRecognition,TestDataset

In [2]:
# config.py 里面定义了 emotion2number 和 intent2number ， number2emotion ， number2intent
# 这四个映射是为了把intent和emotion 和数字之间相互映射，才能进行学习
print(cf.emotion2number)
print(cf.number2emotion)

{'neutral': 4, 'disgust': 1, 'happy': 3, 'anger': 0, 'sad': 5, 'fear': 2, 'surprise': 6}
{4: 'neutral', 1: 'disgust', 3: 'happy', 0: 'anger', 5: 'sad', 2: 'fear', 6: 'surprise'}


In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 要加载具体的模型，模型结构上的所有parms都要相同
# text_head, text_layers, audio_head , audio_layers , video_head,video_layers
# 每次保存模型都会输出parms，直接复制粘贴到这里

params = {'audio_head': 16, 'audio_layers': 2, 'batch_size': 32, 'epochs': 5, 'learning_rate': 0.0001, 'text_head': 16, 'text_layers': 2, 'video_head': 16, 'video_layers': 2}
model_name ="10_31_xmodel.pth"

model_save_path = f"{cf.BEST_MODEL_SAVE_DIR}{model_name}"

TEXT_DIM,AUDIO_DIM,VIDEO_DIM = cf.get_feature_dim()

model = MultimodalEmotionRecognition(
        text_dim=TEXT_DIM, audio_dim=AUDIO_DIM, video_dim=VIDEO_DIM, 
        intent_classes=8, emotion_classes=7, 
        text_head=params['text_head'], text_layers=params['text_layers'],  # 文本 Transformer 参数
        audio_head=params['audio_head'], audio_layers=params['audio_layers'],  # 音频 Transformer 参数
        video_head=params['video_head'], video_layers=params['video_layers'],  # 视频 Transformer 参数
).to(DEVICE)

model_dict = torch.load(model_save_path)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [10]:
# 在测试集上进行推理, 并且保存结果为submission.csv

SAVE_DIR = cf.BEST_MODEL_SAVE_DIR #测试结果保存目录，可以更改，如果不更改，就默认是保存模型的目录
result_name = "submission_10.30.csv"


# -------don't modify----------------

test_dataset = TestDataset()
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

all_filename = []
emo_pred = []
int_pred = []

with torch.no_grad():
    for text, audio,video,filename in test_loader:
        text,audio,video = text.to(DEVICE),audio.to(DEVICE),video.to(DEVICE)
        intent_output, emotion_output = model(text,audio,video)
        _, predicted_intent = torch.max(intent_output, 1)
        _, predicted_emotion = torch.max(emotion_output, 1)
        # 根据映射关系转换为intent和emotion字符串
        predicted_intent_str = [cf.number2intent[x.item()] for x in predicted_intent]
        predicted_emotion_str = [cf.number2emotion[x.item()] for x in predicted_emotion]
        emo_pred.append(predicted_emotion_str)
        int_pred.append(predicted_intent_str)
        all_filename.append(filename)

# 把结果保存为csv
emo_pred = np.concatenate(emo_pred)
int_pred = np.concatenate(int_pred)
all_filename = np.concatenate(all_filename)

# 创建DataFrame并保存为CSV文件
submission_df = pd.DataFrame({
        'filename': all_filename,
        'emo_pred': emo_pred,
        'int_pred': int_pred
})


save_dictory = f"{SAVE_DIR}{result_name}"

submission_df.to_csv(save_dictory, index=False)
print(f"Results saved to {save_dictory}")

Results saved to /home/ubuntu/zlb/MEIJU/model/save/submission_10.30.csv
